In [43]:
!pip install simpy plotly

### Goal
- clean up 
- logging
- merge system metrics

In [44]:
import simpy
import datetime
import pandas as pd
import plotly.express as px

# new imports
import logging
from enum import Enum

In [45]:
class Metric(Enum):
    RW = "Requests Waiting"
    BS = "Busy Slots"

In [46]:
log_filename = "logs-5.log"
mainLogger = logging.getLogger()
mainLogger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_filename, mode='w')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
mainLogger.addHandler(fhandler)
mainLogger.setLevel(logging.DEBUG)
mainLogger.debug("test")

In [47]:
class User():
    def __init__(self, id, env, res, monitor):
        self.id = id
        self.env = env
        self.res = res
        self.monitor = monitor
        self.taskid = 1
        self.name = f"User-{id}"
        mainLogger.info(f"creating user {self.name}")
        #print(f"creating user {self.name}")
        # Start the run process everytime an instance is created.
        # create itself as a processs
        self.action = env.process(self.run())

    def run(self):
        while True:
            taskname = "process_task"
            self.taskid += 1
            self.monitor.report_start(self.name, taskname, self.taskid)
            #print(f"{self.name} starts task at %d" % self.env.now)
            mainLogger.debug(f"{self.name} starts task at %d" % self.env.now)
            task_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.process_task(task_duration))
            #print(f"{self.name} ends task at %d" % self.env.now)
            mainLogger.debug(f"{self.name} ends task at %d" % self.env.now)
            self.monitor.report_stop(self.name, taskname, self.taskid)
     
            # The charge process has finished and
            # we can start driving again.
            #print(f"{self.name} starts verification at %d" % self.env.now)
            mainLogger.debug(f"{self.name} starts verification at %d" % self.env.now)
            verification_duration = 2
            yield self.env.timeout(verification_duration)
            #print(f"{self.name} ends verification at %d" % self.env.now)
            mainLogger.debug(f"{self.name} ends verification at %d" % self.env.now)

    def process_task(self, duration):
        with self.res.request() as req:
            yield req
            yield self.env.timeout(duration)


In [48]:
class Clock:
    def __init__(self):
        #self.base_epoch = datetime.datetime(2021,9,10,0,0).timestamp()
        self.base_epoch = datetime.datetime.now().timestamp()
        #print(f"Clock created - base {self.base_epoch}")
        mainLogger.info(f"Clock created - base {self.base_epoch}")

    def to_date(self, tick):
        epoch_time = self.base_epoch + tick*60 #mn
        #print(f"Clock {epoch_time}")
        datetime_time = datetime.datetime.fromtimestamp(epoch_time)
        return datetime_time

In [49]:
class UsersMonitor:
    def __init__(self, env, clock):
        self.env = env
        self.start_data = []
        self.stop_data = []
        
    def report_start(self, username, taskname, taskid):
        mark = self.env.now
        self.start_data.append(
            dict(  
                StartMark=mark,
                Start=clock.to_date(mark),
                Username=username,
                Task=taskname,
                TaskId=taskid
            )
        )       
        
    def report_stop(self, username, taskname, taskid):
        mark = self.env.now
        self.stop_data.append(
            dict(  
                FinishMark=mark,
                Finish=clock.to_date(mark),
                Username=username,
                Task=taskname,
                TaskId=taskid
            )
        )
        
                
    def collect(self):
        df_start = pd.DataFrame(self.start_data)
        df_stop = pd.DataFrame(self.stop_data)
        df = pd.merge(df_start, df_stop, how='left', 
                      on = ['Username', 'Task', 'TaskId'])
        df['Duration'] = df['FinishMark'] - df['StartMark']
        return df


In [50]:
def user_generator(size, env, res, user_monitor):
    for i in range(size):
        User(i, env, res, user_monitor).action
        yield env.timeout(5)

In [51]:
class SystemResource(simpy.Resource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        #print(f"create resource with capacity {self.capacity}")
        mainLogger.info(f"create resource with capacity {self.capacity}")

    def request(self, *args, **kwargs):
        #print("request resource at %d" % self._env.now)
        mainLogger.debug("request resource at %d" % self._env.now)
        return super().request(*args, **kwargs)

    def release(self, *args, **kwargs):
        #print("release resource at %d" % self._env.now)
        mainLogger.debug("release resource at %d" % self._env.now)
        return super().release(*args, **kwargs)


In [52]:
# wake up every tick and collect
class SystemMonitoringAgent:
    def __init__(self, env, res, clock):
        self.env = env
        self.res = res
        self.clock = clock
        #print("creating agent")
        mainLogger.info("creating agent")
        #self.occupied_slots_data = []
        #self.requests_waiting_data = []
        self.data = []
        # this will be used as a process
        self.action = env.process(self.run())

    def run(self):
        while True:

            mark = self.env.now
            occupied_slots = res.count
            requests_waiting = len(res.queue)
            
            #print(f"{occupied_slots} occupied slots at %d" % mark)
            #print(f"{requests_waiting} requests waiting at %d" % mark)
            mainLogger.info(f"{occupied_slots} occupied slots at %d" % mark)
            mainLogger.info(f"{requests_waiting} requests waiting at %d" % mark)

            #self.occupied_slots_data.append(
            self.data.append(
                dict(  
                    Mark=mark,
                    Timestamp=clock.to_date(mark),
                    Metric=Metric.BS.value,
                    Value=occupied_slots
                )
            )       
            #self.requests_waiting_data.append(
            self.data.append(
                dict(  
                    Mark=mark,
                    Timestamp=clock.to_date(mark),
                    Metric=Metric.RW.value,
                    Value=requests_waiting
                )
            ) 
            
            tick_duration = 1
            yield self.env.timeout(tick_duration)

        
                
    def collect(self):
        #return { 'occupied_slots': pd.DataFrame(self.occupied_slots_data), 
        #        'requests_waiting': pd.DataFrame(self.requests_waiting_data),
        #        'users_count': pd.DataFrame(self.users_count_data)}
        return pd.DataFrame(self.data)


## Configuration

In [53]:
# sim
sim_duration = 200
nb_users = 20
resource_capacity = 5 # 1 -> 5

mainLogger.setLevel(logging.INFO)


## Main process

In [54]:
env = simpy.Environment()
clock = Clock()
res = SystemResource(env, capacity=resource_capacity)
user_monitor = UsersMonitor(env, clock)
user_gen = env.process(user_generator(nb_users, env, res, user_monitor))
res_agent = SystemMonitoringAgent(env, res, clock)

env.run(until=sim_duration)

In [55]:
#dfs = res_agent.collect()
#print(dfs)
df_system = res_agent.collect()
display(df_system)

,Mark,Metric,Timestamp,Value
0,0,Busy Slots,2021-09-12 11:25:51.294027,0
1,0,Requests Waiting,2021-09-12 11:25:51.294027,0
2,1,Busy Slots,2021-09-12 11:26:51.294027,1
3,1,Requests Waiting,2021-09-12 11:26:51.294027,0
4,2,Busy Slots,2021-09-12 11:27:51.294027,1
5,2,Requests Waiting,2021-09-12 11:27:51.294027,0
6,3,Busy Slots,2021-09-12 11:28:51.294027,1
7,3,Requests Waiting,2021-09-12 11:28:51.294027,0
8,4,Busy Slots,2021-09-12 11:29:51.294027,1
9,4,Requests Waiting,2021-09-12 11:29:51.294027,0


## Occupied slots

In [56]:
#df = dfs['occupied_slots']
df = df_system[ df_system.Metric == Metric.BS.value ]
fig = px.bar(df, x='Timestamp', y='Value')
fig.show()

## Requests waiting

In [57]:
#df = dfs['requests_waiting']
df = df_system[ df_system.Metric == Metric.RW.value ]
fig = px.bar(df, x='Timestamp', y='Value')
fig.show()

## Users activity

In [58]:
df_users = user_monitor.collect()
display(df_users)

,Start,StartMark,Task,TaskId,Username,Finish,FinishMark,Duration
0,2021-09-12 11:25:51.294027,0,process_task,2,User-0,2021-09-12 11:30:51.294027,5.0,5.0
1,2021-09-12 11:30:51.294027,5,process_task,2,User-1,2021-09-12 11:35:51.294027,10.0,5.0
2,2021-09-12 11:32:51.294027,7,process_task,3,User-0,2021-09-12 11:37:51.294027,12.0,5.0
3,2021-09-12 11:35:51.294027,10,process_task,2,User-2,2021-09-12 11:40:51.294027,15.0,5.0
4,2021-09-12 11:37:51.294027,12,process_task,3,User-1,2021-09-12 11:42:51.294027,17.0,5.0
5,2021-09-12 11:39:51.294027,14,process_task,4,User-0,2021-09-12 11:44:51.294027,19.0,5.0
6,2021-09-12 11:40:51.294027,15,process_task,2,User-3,2021-09-12 11:45:51.294027,20.0,5.0
7,2021-09-12 11:42:51.294027,17,process_task,3,User-2,2021-09-12 11:47:51.294027,22.0,5.0
8,2021-09-12 11:44:51.294027,19,process_task,4,User-1,2021-09-12 11:49:51.294027,24.0,5.0
9,2021-09-12 11:45:51.294027,20,process_task,2,User-4,2021-09-12 11:50:51.294027,25.0,5.0


In [59]:
fig = px.timeline(df_users, x_start="Start", x_end="Finish", y="Username")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

## Users activity distribution

In [60]:
df_users[["Duration"]].describe()

,Duration
count,183.000000
mean,13.737705
std,5.044532
min,5.000000
25%,9.000000
50%,17.000000
75%,18.000000
max,18.000000


In [61]:
mainLogger.debug(f"max={max}")
fig = px.histogram(df_users, x="Duration", nbins=20)
fig.show()

In [62]:
# when starts waiting - more or less 8 users
# as much as processes - more or less 13 users

# Logs

In [63]:
with open(log_filename) as log:
            print(log.read())

2021-09-12 11:25:51,229 - root - DEBUG - test
2021-09-12 11:25:51,294 - root - INFO - Clock created - base 1631438751.294027
2021-09-12 11:25:51,294 - root - INFO - create resource with capacity 5
2021-09-12 11:25:51,294 - root - INFO - creating agent
2021-09-12 11:25:51,294 - root - INFO - creating user User-0
2021-09-12 11:25:51,295 - root - INFO - 0 occupied slots at 0
2021-09-12 11:25:51,295 - root - INFO - 0 requests waiting at 0
2021-09-12 11:25:51,295 - root - INFO - 1 occupied slots at 1
2021-09-12 11:25:51,295 - root - INFO - 0 requests waiting at 1
2021-09-12 11:25:51,295 - root - INFO - 1 occupied slots at 2
2021-09-12 11:25:51,296 - root - INFO - 0 requests waiting at 2
2021-09-12 11:25:51,296 - root - INFO - 1 occupied slots at 3
2021-09-12 11:25:51,296 - root - INFO - 0 requests waiting at 3
2021-09-12 11:25:51,296 - root - INFO - 1 occupied slots at 4
2021-09-12 11:25:51,296 - root - INFO - 0 requests waiting at 4
2021-09-12 11:25:51,296 - root - INFO - creating user Use